# Collaborative Filtering (CS246)

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
import os

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [4]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

zsh:1: command not found: wget
unzip:  cannot find or open ngrok-stable-linux-amd64.zip, ngrok-stable-linux-amd64.zip.zip or ngrok-stable-linux-amd64.zip.ZIP.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/jaeyong/opt/anaconda3/lib/python3.8/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/Users/jaeyong/opt/anaconda3/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/Users/jaeyong/opt/anaconda3/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/jaeyong/opt/anaconda3/lib/python3.8/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [6]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("./dataset/MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("./dataset/MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("./dataset/MovieLens.item", header=False, schema=schema_items)

In [7]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [8]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [9]:
# YOUR CODE HERE
movie_counts = training.groupBy("item_id")\
                    .agg(count("*").alias("MovieCount"))\
                    .sort(desc("MovieCount"))

movie_counts.count()
#traing 영화 개수는 1650개

1650

Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [10]:
movie_counts_test = items.groupBy("item_id")\
                    .agg(count("*").alias("MovieCount"))\
                    .sort(desc("MovieCount"))

movie_counts_test.count()
#test 영화 개수는 1650개

1682

Now compute the RMSE on the test dataset.


In [11]:
movie_ratings = training.groupBy('item_id')\
                        .agg(mean('rating').alias('MovieRating'))\
                        .sort(desc('MovieRating'))

movie_ratings.show()
#영화별 평점(평균) 내림차순

+-------+-----------------+
|item_id|      MovieRating|
+-------+-----------------+
|   1122|              5.0|
|   1653|              5.0|
|   1201|              5.0|
|   1293|              5.0|
|   1467|              5.0|
|   1599|              5.0|
|   1189|              5.0|
|   1500|              5.0|
|   1449|4.714285714285714|
|   1367|4.666666666666667|
|    408|4.559139784946237|
|    169|4.525773195876289|
|    850|              4.5|
|   1642|              4.5|
|   1594|              4.5|
|    318|4.489451476793249|
|    483|4.441025641025641|
|     64|4.417040358744394|
|     12|4.398104265402844|
|     50|4.359504132231405|
+-------+-----------------+
only showing top 20 rows



At this point, you can use the trained model to produce the top-K recommendations for each user.

In [12]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = training.rdd
ratings = spark.createDataFrame(lines)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [13]:
lines.take(3)

[Row(user_id=1, item_id=1, rating=5, timestamp=874965758),
 Row(user_id=1, item_id=2, rating=3, timestamp=876893171),
 Row(user_id=1, item_id=3, rating=4, timestamp=878542960)]

In [14]:
# YOUR CODE HERE
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1185899533373342


In [15]:
# YOUR CODE HERE
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)
# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

In [16]:
movieRecs.take(3)

[Row(item_id=1580, recommendations=[Row(user_id=300, rating=1.5532881021499634), Row(user_id=107, rating=1.5013761520385742), Row(user_id=261, rating=1.462632656097412), Row(user_id=78, rating=1.456456184387207), Row(user_id=39, rating=1.4514678716659546), Row(user_id=626, rating=1.4084563255310059), Row(user_id=97, rating=1.3475079536437988), Row(user_id=366, rating=1.3370845317840576), Row(user_id=362, rating=1.311409592628479), Row(user_id=258, rating=1.2446609735488892)]),
 Row(item_id=471, recommendations=[Row(user_id=337, rating=6.2447710037231445), Row(user_id=180, rating=6.158797740936279), Row(user_id=636, rating=5.850564956665039), Row(user_id=340, rating=5.534483432769775), Row(user_id=335, rating=5.439809322357178), Row(user_id=173, rating=5.363190650939941), Row(user_id=278, rating=5.182185173034668), Row(user_id=252, rating=5.171838760375977), Row(user_id=372, rating=5.147116661071777), Row(user_id=849, rating=5.111682891845703)]),
 Row(item_id=1591, recommendations=[Row(

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!